In [ ]:
from openpyxl import load_workbook
import pandas as pd

In [ ]:


# Charger le fichier Excel
workbook = load_workbook("LinkedIn.xlsx")
sheet = workbook.active  # Accéder à la première feuille (ou spécifier le nom)

# Parcourir les images de la feuille
for image in sheet._images:
    print(f"Image: {image}")
    # Vous pouvez ensuite accéder à l'image elle-même via image.ref et l'enregistrer
    print(image.ref)  # Position de l'image
    # Enregistrer l'image si besoin
    print(type(image.anchor))  # Positionnement de l'image (coordonnée cellule)


In [ ]:
# Lire les données en ignorant les images
data = pd.read_excel("LinkedIn.xlsx", engine='openpyxl')
# Effectuer vos traitements sur le DataFrame


In [ ]:
import re
def split_value(value: str):
    regex = r'Nom du membre(.*)Fonction du membre(.*)Connecté(.*)'
    match = re.match(regex, str(value))
    if match:
        return {"nom": match.group(1).strip(), "fonction": match.group(2).strip(), "connecte": match.group(3).strip()}
    else:
        return {"nom": None, "fonction": None, "connecte": None}


def filter_value(value: str):
    regex = r'Nom du membre(.*)Fonction du membre(.*)Connecté(.*)'
    match = re.match(regex, str(value))
    if match:
        return True
    else:
        return False


In [ ]:
data = data[data["N°"]!="Message"]
data = data[~data["N°"].isna()]
data["filtre"] = data["N°"].apply(filter_value)
data["dict"] = data["N°"].apply(split_value)
data = data[data["filtre"]]
data.info()

In [ ]:
data["Nom"] = data["dict"].apply(lambda x: x["nom"])
data["Fonction"] = data["dict"].apply(lambda x: x["fonction"])
data["Connecté"] = data["dict"].apply(lambda x: x["connecte"])
data.tail(30)

In [ ]:
data2 = data[["Nom", "Fonction", "Connecté"]]
data2.info()

In [ ]:
data2.to_excel("Processed_data.xlsx", index=False)

In [ ]:
import pandas as pd
from openpyxl import load_workbook

# Charger le fichier Excel
wb = load_workbook('LinkedIn.xlsx')
ws = wb.active  # ou wb['nom_de_votre_feuille'] si vous connaissez le nom

# Pour stocker les données
donnees = []

# Supposons que la colonne N° est la première colonne (A)
# Ajustez le range si nécessaire selon votre fichier
for row in ws.iter_rows(min_row=2):  # min_row=2 pour ignorer l'en-tête
    cell = row[0]  # Première colonne (N°)
    
    texte = cell.value  # Le texte visible
    if not filter_value(texte):
        continue

    url = None    
    if cell.hyperlink:
        # Récupérer l'URL du lien hypertexte
        url = cell.hyperlink.target
        
    donnees.append({'texte': texte, 'url': url})

# Créer un DataFrame
df = pd.DataFrame(donnees)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.to_excel("comp.xlsx", index=False)

In [27]:
import pandas as pd
from openpyxl import load_workbook
import re

def split_value(value: str):
    regex = r'Nom du membre(.*)Fonction du membre(.*)Connecté(.*)'
    match = re.match(regex, str(value))
    if match:
        return {"nom": match.group(1).strip(), "fonction": match.group(2).strip(), "connecte": match.group(3).strip()}
    else:
        return {"nom": None, "fonction": None, "connecte": None}


def filter_value(value: str):
    regex = r'Nom du membre(.*)Fonction du membre(.*)Connecté(.*)'
    match = re.match(regex, str(value))
    if match:
        return True
    else:
        return False
    
fichier = "LinkedIn.xlsx" # Chemin du fichier à traîter
    
# TRAITEMENT 1: EXTRACTION DU NOM, DE LA FONCTION ET DU MOMENT DE CONNECTION #
# Lire les données en ignorant les images
data = pd.read_excel(fichier, engine='openpyxl')

data = data[data["N°"]!="Message"]
data = data[~data["N°"].isna()]
data["filtre"] = data["N°"].apply(filter_value)
data = data[data["filtre"]]
data["dict"] = data["N°"].apply(split_value)

data["Nom"] = data["dict"].apply(lambda x: x["nom"])
data["Fonction"] = data["dict"].apply(lambda x: x["fonction"])
data["Connecté"] = data["dict"].apply(lambda x: x["connecte"])

data = data[["Nom", "Fonction", "Connecté"]].reset_index().drop(["index"], axis="columns")


# TRAITEMENT 2: EXTRACTION DU LIEN DE LA PAGE LINKEDIN ET ENREGISTREMENT DU FICHIER
# Charger le fichier Excel
wb = load_workbook(fichier)
ws = wb.active  # ou wb['nom_de_votre_feuille'] si vous connaissez le nom

# Pour stocker les données
donnees = []

# Ajustez le range si nécessaire selon votre fichier
for row in ws.iter_rows(min_row=2):  # min_row=2 pour ignorer l'en-tête
    cell = row[0]  # Première colonne (N°)
    
    texte = cell.value  # Le texte visible
    if not filter_value(texte):
        continue
    nom = split_value(texte)["nom"]

    url = None    
    if cell.hyperlink:
        # Récupérer l'URL du lien hypertexte
        url = cell.hyperlink.target
        
    donnees.append({'Nom': nom, 'Lien': url})

# Créer un DataFrame
df = pd.DataFrame(donnees)

# Joindre les deux dataframes et enregistrer le résultat
# data = data.join(df, on="Nom", how="inner")
data["Lien"] = df["Lien"]
data.to_excel("Processed_data.xlsx", index=False)

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2987 entries, 0 to 2986
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Nom       2987 non-null   object
 1   Fonction  2987 non-null   object
 2   Connecté  2987 non-null   object
 3   Lien      2987 non-null   object
dtypes: object(4)
memory usage: 93.5+ KB


In [29]:
data.head(30)

,Nom,Fonction,Connecté,Lien
0,Arthur Vaillant,Responsable Grands Comptes et Institutionnels,il y a 10 heures,https://www.linkedin.com/in/arthur-vaillant-89...
1,Darly NGUEMA,Diplômée en Finance Audit et Pilotage de la pe...,il y a 23 heures,https://www.linkedin.com/in/darly-nguema-08695...
2,Jean-Christophe CATTEAU,Directeur des Opérations et de la Qualité de S...,il y a 1 jour,https://www.linkedin.com/in/jean-christophe-ca...
3,Tamara Nahon,Née en 366.27 ppm - Associée Team For The Planet,il y a 1 jour,https://www.linkedin.com/in/tamara-nahon/
4,Cyndie Rey,Responsable Recrutement,il y a 1 jour,https://www.linkedin.com/in/cyndie-rey-233971133/
5,Stéphane Camon,Leader de l'écosystème Ma Santé et Directeur T...,il y a 1 jour,https://www.linkedin.com/in/stephanecamon/
6,Patrick Achi,Ancien Premier ministre de la République de Cô...,il y a 1 jour,https://www.linkedin.com/in/patrickachi/
7,Rédoine Kherbache,Learning Business Partner chez Thales,il y a 1 jour,https://www.linkedin.com/in/r%C3%A9doine-kherb...
8,Sylvia de Téglassy,International Executive Search 🎯 Global Execut...,il y a 1 jour,https://www.linkedin.com/in/sylviadeteglassy/
9,Adalbert de Lapresle,Sous Direction de la Maitrise d'Ouvrage,il y a 1 jour,https://www.linkedin.com/in/adalbert-de-lapres...
